In [18]:
import requests
import sys
from tqdm.asyncio import tqdm
import gzip
import csv
import aiohttp
import asyncio
import pandas as pd

df = pd.read_csv("UHC_payers_deduped.csv")

In [15]:
async def get_size(session, url):
    # if "https" not in url: # for Humana only
        # url = "https://developers.humana.com/Resource/DownloadPCTFile" + url

    try:
        async with session.head(url, ssl=False, raise_for_status=True) as r:
            return r.headers.get("Content-Length")
    except:
        pass

In [16]:
async def main():
    async with aiohttp.ClientSession(raise_for_status = True) as session:
        df["size"] = await tqdm.gather(*(get_size(session, url) for url in df["url"]), leave=True, position=0)
        return df



In [19]:
df1 = await main()

100%|██████████| 4329/4329 [00:08<00:00, 502.76it/s]


In [12]:
df1 = df1.sort_values(by="size")

In [13]:
df1.to_csv("UHC_payers_sorted.csv", index=False)